In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

2023-07-28 22:44:31.036168: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-28 22:44:31.043072: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 22:44:31.083093: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 22:44:31.084113: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 22:44:31.841286: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [5]:
l0 = Dense(units=1, input_shape=[1])
model = Sequential([l0])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict([10.0]))
print("Here is what I learned: {}".format(l0.get_weights()))
print(l0)

Epoch 1/500
1/1 [==============================] - 0s 309ms/step - loss: 1.9657
Epoch 2/500
1/1 [==============================] - 0s 6ms/step - loss: 1.6899
Epoch 3/500
1/1 [==============================] - 0s 4ms/step - loss: 1.4700
Epoch 4/500
1/1 [==============================] - 0s 4ms/step - loss: 1.2941
Epoch 5/500
1/1 [==============================] - 0s 7ms/step - loss: 1.1529
Epoch 6/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0390
Epoch 7/500
1/1 [==============================] - 0s 4ms/step - loss: 0.9467
Epoch 8/500
1/1 [==============================] - 0s 5ms/step - loss: 0.8715
Epoch 9/500
1/1 [==============================] - 0s 5ms/step - loss: 0.8096
Epoch 10/500
1/1 [==============================] - 0s 6ms/step - loss: 0.7584
Epoch 11/500
1/1 [==============================] - 0s 9ms/step - loss: 0.7157
Epoch 12/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6796
Epoch 13/500
1/1 [==============================] - 0s 8ms/

In [4]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets


In [5]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

2023-07-28 15:23:32.098263: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-28 15:23:32.098296: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-28 15:23:32.099007: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: saved_model/1
2023-07-28 15:23:32.099409: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-07-28 15:23:32.099418: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: saved_model/1
2023-07-28 15:23:32.100490: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-07-28 15:23:32.100800: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-07-28 15:23:32.117817: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: saved_model/1
2023-07-28 15:23

In [6]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

1080

In [7]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_dense_input:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]
[[18.987448]]
